## Modeling and Validation / CrossValidation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [4]:
data = pd.read_parquet('data/processed_data.parquet')

print(data.head()) 

            counter_id              counter_name    site_id  \
0  100007049-102007049  28 boulevard Diderot E-O  100007049   
1  100007049-102007049  28 boulevard Diderot E-O  100007049   
2  100007049-102007049  28 boulevard Diderot E-O  100007049   
3  100007049-102007049  28 boulevard Diderot E-O  100007049   
4  100007049-102007049  28 boulevard Diderot E-O  100007049   

              site_name  bike_count                date  \
0  28 boulevard Diderot         0.0 2020-09-01 02:00:00   
1  28 boulevard Diderot         1.0 2020-09-01 03:00:00   
2  28 boulevard Diderot         0.0 2020-09-01 04:00:00   
3  28 boulevard Diderot         4.0 2020-09-01 15:00:00   
4  28 boulevard Diderot         9.0 2020-09-01 18:00:00   

  counter_installation_date         coordinates counter_technical_id  \
0                2013-01-18  48.846028,2.375429          Y2H15027244   
1                2013-01-18  48.846028,2.375429          Y2H15027244   
2                2013-01-18  48.846028,2.375429   